# *LinearSolve* Package

## Contents

- [TBD](#tbd)

LinearSolve.jl is a unified interface for the linear solving packages of Julia. It interfaces with other packages of the Julia ecosystem to make it easy to test alternative solver packages and pass small types to control algorithm swapping. It also interfaces with the ModelingToolkit.jl world of symbolic modeling to allow for automatically generating high-performance code.

## But hWhy?

In [17]:
using LinearAlgebra

In [22]:
A = [1. 1. 3.
     4. 5. 6.
     7. 8. 9.]
b1 = [1., 2., 3.]
b2 = [1., 0., 1.];

In [28]:
@time A\b1; A\b2;
@time lu!(A); A\b1; A\b2;

  0.000029 seconds (6 allocations: 304 bytes)
  0.000022 seconds (3 allocations: 112 bytes)


In [43]:
A = [1. 1. 3.
     4. 5. 6.
     7. 8. 9.]
b1 = [1., 2., 3.]
b2 = [1., 0., 1.]
x1 = similar(b1);

In [44]:
@time ldiv!(x1, lu(A), b1)

  0.000029 seconds (5 allocations: 256 bytes)


3-element Vector{Float64}:
  1.4802973661668746e-16
 -1.2952601953960153e-16
  0.3333333333333333

## Solving Linear Systems in Julia

A linear system $Au=b$ is specified by defining an `AbstractMatrix A`, or by providing a matrix-free operator for performing `A*x` operations via the function `A(u,p,t)` out-of-place and `A(du,u,p,t)` for in-place. For the sake of simplicity, this tutorial will only showcase concrete matrices.

The following defines a matrix and a `LinearProblem` which is subsequently solved by the default linear solver.

In [3]:
using LinearSolve

A = rand(4, 4)
b = rand(4)
prob = LinearProblem(A, b)
sol = solve(prob)
sol.u

4-element Vector{Float64}:
  1.5659200410393903
 -5.486262867417643
 -4.752664199154205
 10.689944190766758

Note that `solve(prob)` is equivalent to `solve(prob,nothing)` where nothing denotes the choice of the default linear solver. This is equivalent to the Julia built-in `A\b`, where the solution is recovered via `sol.u`. The power of this package comes into play when changing the algorithms. For example, *Krylov.jl* has some nice methods like *GMRES* which can be faster in some cases. With *LinearSolve.jl*, there is one interface and changing linear solvers is simply the switch of the algorithm choice:

In [45]:
sol = solve(prob, KrylovJL_GMRES())
sol.u

4-element Vector{Float64}:
  1.5659200410393932
 -5.48626286741767
 -4.752664199154221
 10.6899441907668

## Avoiding Refactorization

In [46]:
using LinearSolve

n = 4
A = rand(n, n)
b1 = rand(n);
b2 = rand(n);
prob = LinearProblem(A, b1)

linsolve = init(prob)
sol1 = solve!(linsolve)

retcode: Default
u: 4-element Vector{Float64}:
 -0.46103645603353366
  0.05660653433983502
  0.9790736146374841
  0.5449291529716783

In [47]:
linsolve.b = b2
sol2 = solve!(linsolve)

sol2.u

4-element Vector{Float64}:
  0.9879549541863708
  0.49121666125626523
 -0.579211416259471
 -0.3970125880491276